El objetivo es garantizar que las empresas reciban una facturación adecuada por los servicios prestados por las empresas de mensajería en cuanto a entrega de bienes en mensajeria B2B

In [1]:
#comencemos importando las librerias necesarias de python y el conjunto de datos
import pandas as pd

order_report = pd.read_csv('Order Report.csv')
invoice = pd.read_csv('Invoice.csv')
courier_company_rates = pd.read_csv('Courier Company - Rates.csv')
pincodes = pd.read_csv('pincodes.csv')
sku_master = pd.read_csv('SKU Master.csv')

In [2]:
print(order_report.head())
print(sku_master.head())
print(pincodes.head())
print(invoice.head())
print(courier_company_rates.head())

   ExternOrderNo            SKU  Order Qty  Unnamed: 3  Unnamed: 4
0     2001827036  8904223818706        1.0         NaN         NaN
1     2001827036  8904223819093        1.0         NaN         NaN
2     2001827036  8904223819109        1.0         NaN         NaN
3     2001827036  8904223818430        1.0         NaN         NaN
4     2001827036  8904223819277        1.0         NaN         NaN
             SKU  Weight (g)  Unnamed: 2  Unnamed: 3  Unnamed: 4
0  8904223815682         210         NaN         NaN         NaN
1  8904223815859         165         NaN         NaN         NaN
2  8904223815866         113         NaN         NaN         NaN
3  8904223815873          65         NaN         NaN         NaN
4  8904223816214         120         NaN         NaN         NaN
   Warehouse Pincode  Customer Pincode Zone  Unnamed: 3  Unnamed: 4
0             121003            507101    d         NaN         NaN
1             121003            486886    d         NaN         NaN
2   

In [3]:
#ahora comprobemos si hay valores vacios en las tablas
print(order_report.isnull().sum())
print(sku_master.isnull().sum())
print(pincodes.isnull().sum())
print(invoice.isnull().sum())
print(courier_company_rates.isnull().sum())

ExternOrderNo      0
SKU                0
Order Qty          0
Unnamed: 3       400
Unnamed: 4       400
dtype: int64
SKU            0
Weight (g)     0
Unnamed: 2    66
Unnamed: 3    66
Unnamed: 4    66
dtype: int64
Warehouse Pincode      0
Customer Pincode       0
Zone                   0
Unnamed: 3           124
Unnamed: 4           124
dtype: int64
AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64
fwd_a_fixed         0
fwd_a_additional    0
fwd_b_fixed         0
fwd_b_additional    0
fwd_c_fixed         0
fwd_c_additional    0
fwd_d_fixed         0
fwd_d_additional    0
fwd_e_fixed         0
fwd_e_additional    0
rto_a_fixed         0
rto_a_additional    0
rto_b_fixed         0
rto_b_additional    0
rto_c_fixed         0
rto_c_additional    0
rto_d_fixed         0
rto_d_additional    0
rto_e_fixed         0
rto_e_a

In [4]:
#ahora limpiemos los datos

#removamos columnas sin nombre del dataframe order_report
order_report = order_report.drop(columns = ['Unnamed: 3', 'Unnamed: 4'])
sku_master = sku_master.drop(columns = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])
pincodes = pincodes.drop(columns = ['Unnamed: 3', 'Unnamed: 4'])

In [5]:
#ahora unimos order_report y sku_master basado en la columna SKU que tienen en comun
merged_data = pd.merge(order_report, sku_master, on = 'SKU')

In [6]:
#cambiemos el nombre de la columna ExternOrderNo a Order ID
merged_data = merged_data.rename(columns = {'ExternOrderNo': 'Order ID'})

In [7]:
#ahora unamos los datos de factura de mensajeria y mapeo de codigo PIN
abc_courier = pincodes.drop_duplicates(subset = ['Customer Pincode'])
courier_abc = invoice[['Order ID', 'Customer Pincode', 'Type of Shipment']]
pincodes = courier_abc.merge(abc_courier, on = 'Customer Pincode')

In [8]:
pincodes

,Order ID,Customer Pincode,Type of Shipment,Warehouse Pincode,Zone
0,2001806232,507101,Forward charges,121003,d
1,2001806273,486886,Forward charges,121003,d
2,2001806408,532484,Forward charges,121003,d
3,2001806458,143001,Forward charges,121003,b
4,2001807012,515591,Forward charges,121003,d
...,...,...,...,...,...
119,2001811039,302020,Forward charges,121003,b
120,2001811305,302020,Forward charges,121003,b
121,2001812941,325207,Forward charges,121003,b
122,2001809383,303702,Forward and RTO charges,121003,b


In [9]:
#ahora unimos los codigos PIN con el dataframe original anteriormente unido
final_merged = merged_data.merge(pincodes, on = 'Order ID')

In [10]:
#ahora calculemos el peso en kilogramos dividiendo Weight(g) del dataframe
#'final_merged' por 1000
final_merged['Weights (kgs)'] = final_merged['Weight (g)'] / 1000

In [11]:
#ahora calculemos las weight labs
def weight_slab(weight):
    i = round(weight % 1, 1)
    if i == 0.0:
        return weight
    elif i > 0.5:
        return int(weight) > 1.0
    else:
        return int(weight) + 0.5
    
final_merged['Weight Slab (kg)'] = final_merged['Weights (kgs)'].apply(weight_slab)
invoice['Weight slab charger by Courier Company'] = (invoice['Charged Weight']).apply(weight_slab)

In [12]:
#ahora renombremos las columnas para preparar el dataframe final deseado
invoice = invoice.rename(columns = {'Zone': 'Delivery zone charged by Courier company'})
final_merged = final_merged.rename(columns = {'Zone': 'Delivery zone as per ABC'})
final_merged = final_merged.rename(columns = {'Weight Slab (kg)': 'Weight slab as per ABC'})

In [14]:
#ahora calculemos los cargos esperados
total_cargos_esperados = []

for i, row in final_merged.iterrows():
    fwd_category = 'fwd_' + row['Delivery zone as per ABC']
    fwd_fixed = courier_company_rates.at[0, fwd_category + '_fixed']
    fwd_additional = courier_company_rates.at[0, fwd_category + '_additional']
    rto_category = 'rto_' + row['Delivery zone as per ABC']
    rto_fixed = courier_company_rates.at[0, rto_category + '_fixed']
    rto_additional = courier_company_rates.at[0, rto_category + '_additional']

    weight_slab = row['Weight slab as per ABC']

    if row['Type of Shipment'] == 'Forward charges': 
        additional_weights = max(0, (weight_slab - 0.5) / 0.5)
        total_cargos_esperados.append(fwd_fixed + additional_weights * fwd_additional)
    elif row['Type of Shipment'] == 'Forward and RTO charges':
        additional_weights = max(0, (weight_slab - 0.5) / 0.5)
        total_cargos_esperados.append(fwd_fixed + additional_weights * (fwd_additional + rto_additional))
    else:
        total_cargos_esperados.append(0)
final_merged['Expected charge as per ABC'] = total_cargos_esperados

In [18]:
final_merged.head()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery zone as per ABC,Weights (kgs),Weight slab as per ABC,Expected charge as per ABC
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.127,0.5,56.6
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.150,0.5,56.6
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.100,0.5,56.6
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.165,0.5,56.6
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.350,0.5,56.6


En el codigo anterior, iteramos a través de cada fila del DataFrame 'final_merged' para calcular los cargos esperados basados en las tarifas de ABC. Recuperamos las tasas y parámetros necesarios, como los cargos fijos y recargos por nivel de peso para envíos de ida y vuelta, basados en el área de entrega.

Luego determinamos la escala de peso para cada fila. Si el tipo de envío es 'Cargos de ida', calculamos el peso adicional más allá de la escala de peso básica (0.5 KG) y aplicamos los cargos adicionales correspondientes. Para los envíos de "Cargos de ida y vuelta", consideramos cargos adicionales por término y componentes de ida y vuelta.

Finalmente, almacenamos los cargos esperados calculados en la columna "Cargos esperados según ABC" del DataFrame "final_merged". Esto nos permite comparar los cargos esperados con los cargos facturados para analizar la precisión de los cargos de la empresa de mensajería.

In [21]:
#ahora unamos este dataframe con la facturacion de mensajeria para obtener el
#dataframe final
df_unido = final_merged.merge(invoice, on = 'Order ID')

In [22]:
df_unido.head()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode_x,Type of Shipment_x,Warehouse Pincode_x,Delivery zone as per ABC,Weights (kgs),Weight slab as per ABC,Expected charge as per ABC,AWB Code,Charged Weight,Warehouse Pincode_y,Customer Pincode_y,Delivery zone charged by Courier company,Type of Shipment_y,Billing Amount (Rs.),Weight slab charger by Courier Company
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.127,0.5,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,False
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.150,0.5,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,False
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.100,0.5,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,False
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.165,0.5,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,False
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.350,0.5,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,False


In [24]:
#ahora calculemos las diferencias en los cargos y los cargos esperados para cada orden
df_diff = df_unido
df_diff['Diferencia (Rs.)'] = df_diff['Billing Amount (Rs.)'] - df_diff['Expected charge as per ABC']
df_final = df_diff[['Order ID', 'Diferencia (Rs.)', 'Expected charge as per ABC']]

In [25]:
df_final.head()

,Order ID,Diferencia (Rs.),Expected charge as per ABC
0,2001827036,61.3,56.6
1,2001827036,61.3,56.6
2,2001827036,61.3,56.6
3,2001827036,61.3,56.6
4,2001827036,61.3,56.6


Ahora resumiremos la precision de la mensajeria B2B basandonos en los cargos cobrados y los cargos esperados.

In [26]:
#calculamos el total de ordenes en cada categoria
total_cargos_correctos = len(df_final[df_final['Diferencia (Rs.)'] == 0])
total_cargos_excedentes = len(df_final[df_final['Diferencia (Rs.)'] > 0])
total_cargos_faltantes = len(df_final[df_final['Diferencia (Rs.)'] < 0])

#calculamos la cantidad total en cada categoria
cantidad_excedente = abs(df_final[df_final['Diferencia (Rs.)'] > 0]['Diferencia (Rs.)'].sum())
cantidad_faltante = df_final[df_final['Diferencia (Rs.)'] < 0]['Diferencia (Rs.)'].sum()
cantidad_correctamente_cargada = df_final[df_final['Diferencia (Rs.)'] == 0]['Expected charge as per ABC'].sum()

#creamos un nuevo dataframe para resumir
data_resumida = {'Description': ['Total orders where ABC has been correctly charged',
                                 'Total orders where ABC has been overcharged',
                                 'Total orders where ABC has been undercharged'],
                'Count': [total_cargos_correctos, total_cargos_excedentes, total_cargos_faltantes],
                'Amount (Rs.)': [cantidad_correctamente_cargada, cantidad_excedente, cantidad_faltante]}
df_resumido = pd.DataFrame(data_resumida)

In [27]:
df_resumido

,Description,Count,Amount (Rs.)
0,Total orders where ABC has been correctly charged,12,507.6
1,Total orders where ABC has been overcharged,382,34031.1
2,Total orders where ABC has been undercharged,7,-165.2


In [28]:
#visualicemos la proporcion de los errores y aciertos de los cargos y cantidades
#en un grafico de torta
import plotly.graph_objects as go
fig = go.Figure(data = go.Pie(labels = df_resumido['Description'],
                              values = df_resumido['Count'],
                              textinfo = 'label+percent',
                              hole = 0.4))
fig.update_layout(title = 'Proportion')
fig.show()

Asi es como podemos analizar la precision de la facturacion B2B utilizando Python como lenguaje de programacion.

El analisis de precision de la facturacion de mensajeria B2B se centra en evaluar la exactitud de las tarifas cobradas por las empresas de mensajeria para la entrega de bienes en transacciones B2B. 

En estos casos, nuestro objetivo es garantizar que las empresas sean facturadas de manera apropiada por los servicios brindados por las empresas de mensajeria.